In [ ]:
import json
from urllib import parse
from io import BytesIO
from config import *
import time
import requests
from PIL import Image
from fake_useragent import UserAgent

# 禁用安全请求警告
from urllib3 import disable_warnings
from urllib3.exceptions import InsecureRequestWarning
disable_warnings(InsecureRequestWarning)

session = requests.session()
session.verify = False

ua = UserAgent(verify_ssl=False)
headers = {
    "User-Agent": ua.random,
    "Host":"kyfw.12306.cn",
    "Referer":"https://kyfw.12306.cn/otn/passport?redirect=/otn/"
}

def login():
    # 打开登录页面
    url = "https://kyfw.12306.cn/otn/login/init"
    session.get(url, headers=headers)
    # 发送验证码
    if captcha()==True:
        #time.sleep(1)
        # 发送登录信息
        data = {
            "username":"sailant",
            "password":"xu",
            "appid":"otn"
        }
        url = "https://kyfw.12306.cn/passport/web/login"
        response = session.post(url, headers=headers, data=data)
        #下面返回的不是json数据，具奇怪，谁可以解之
        try:
            niubi=json.loads(response.text)
            if niubi["result_code"]=="5":
                print("12306你牛逼，草拟吗"+niubi["result_message"])
                print("这一回返回的是json，并且返回值为5,12306肯定存在校验机制")
            else:
                print("返回的是json，并且不是result为5")
          
        except:
            print('又返回一窜html')
        
        # 这里面返回的不是json了，而是一堆html 文件，很诡异，所以json.loads不了
        data = {
            "appid":"otn"
        }
        url = "https://kyfw.12306.cn/passport/web/auth/uamtk"
        response = session.post(url, headers=headers, data=data)
        if response.status_code == 200:
            fanhui=json.loads(response.text)
            if fanhui['result_code']==1:
                return False
            elif fanhui['result_code']==0:
                print("嘿嘿，还是搞定了吧,登录成功")
                return True
        
def captcha():
    data = {
        "login_site": "E",
        "module": "login",
        "rand": "sjrand",
        "0.7065754052770323":""
    }

    # 获取验证码
    param = parse.urlencode(data)
    url = "https://kyfw.12306.cn/passport/captcha/captcha-image?{}".format(param)
    response = session.get(url, headers=headers)
    if response.status_code == 200:
        file = BytesIO(response.content)
        img = Image.open(file)
        img.show()

    positions = input("请输入验证码: ")
    # 发送验证码
    data = {
        "answer": positions,
        "login_site": "E",
        "rand": "sjrand"
    }

    url = "https://kyfw.12306.cn/passport/captcha/captcha-check"
    response = session.post(url, headers=headers, data=data)
    if response.status_code == 200:
        result=json.loads(response.text)  #这里把str转换为 dict
        if result["result_code"] =="4":
       
            return True
        else:

            return False

def enquire():
      
        query_data={"leftTicketDTO.train_date":"2018-01-30",
                    "leftTicketDTO.from_station":"WGH",
                    "leftTicketDTO.to_station":"UUH",
                    "purpose_codes":"ADULT"
                   }
        param=parse.urlencode(query_data)
        url='https://kyfw.12306.cn/otn/leftTicket/queryZ?{}'.format(param)
        print(url)
        response=session.get(url,headers=headers)
        if response.status_code==200:
            
            print(response.text)
            
        
   

if __name__ == "__main__":
    if login()==1:
        print("来走起开始抢票")
        enquire()
    else:
        print("前面的json又tm卡死了")
   
  
